In [1]:
from game import TheGang

In [2]:
environment = TheGang()

In [3]:
environment.reset()

[Card(value=3, suit=1), Card(value=13, suit=1)]
Rank.HIGH_CARD
None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
for hand in environment.hands:
    print(hand)

5♣, 3♠
7♣, 4♣
K♦, 9♥
8♣, 5♠


In [ ]:
environment.step([0, 0, 0, 0])

{'state': [HandFeatures(rank=<Rank.STRAIGHT: 4>, two_pair_potential=<Potential.POSSIBLE: 1>, three_of_a_kind_potential=<Potential.POSSIBLE: 1>, straight_potential=<Potential.HAS_RANK: 3>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.IMPOSSIBLE: 0>, four_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_flush_potential=<Potential.IMPOSSIBLE: 0>),
  HandFeatures(rank=<Rank.TWO_PAIR: 2>, two_pair_potential=<Potential.HAS_RANK: 3>, three_of_a_kind_potential=<Potential.ONE_CARD_AWAY: 2>, straight_potential=<Potential.POSSIBLE: 1>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.ONE_CARD_AWAY: 2>, four_of_a_kind_potential=<Potential.POSSIBLE: 1>, straight_flush_potential=<Potential.POSSIBLE: 1>),
  HandFeatures(rank=<Rank.HIGH_CARD: 0>, two_pair_potential=<Potential.POSSIBLE: 1>, three_of_a_kind_potential=<Potential.POSSIBLE: 1>, straight_potential=<Potential.POSSIBLE: 1>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_pot

In [ ]:
for hand in environment.hands:
    print(hand)

5♣, 3♠, 6♣, 7♦, 4♦
7♣, 4♣, 6♣, 7♦, 4♦
K♦, 9♥, 6♣, 7♦, 4♦
8♣, 5♠, 6♣, 7♦, 4♦


In [ ]:
environment.step([0, 0, 0, 0])

{'state': [HandFeatures(rank=<Rank.STRAIGHT: 4>, two_pair_potential=<Potential.IMPOSSIBLE: 0>, three_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_potential=<Potential.HAS_RANK: 3>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.IMPOSSIBLE: 0>, four_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_flush_potential=<Potential.IMPOSSIBLE: 0>),
  HandFeatures(rank=<Rank.TWO_PAIR: 2>, two_pair_potential=<Potential.HAS_RANK: 3>, three_of_a_kind_potential=<Potential.ONE_CARD_AWAY: 2>, straight_potential=<Potential.IMPOSSIBLE: 0>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.ONE_CARD_AWAY: 2>, four_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_flush_potential=<Potential.IMPOSSIBLE: 0>),
  HandFeatures(rank=<Rank.HIGH_CARD: 0>, two_pair_potential=<Potential.IMPOSSIBLE: 0>, three_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_potential=<Potential.IMPOSSIBLE: 0>, flush_potential=<Potential.IMPOSSIBLE: 0>

In [ ]:
for hand in environment.hands:
    print(hand)

5♣, 3♠, 6♣, 7♦, 4♦, Q♣
7♣, 4♣, 6♣, 7♦, 4♦, Q♣
K♦, 9♥, 6♣, 7♦, 4♦, Q♣
8♣, 5♠, 6♣, 7♦, 4♦, Q♣


In [ ]:
environment.step([0, 0, 0, 0])

{'state': [HandFeatures(rank=<Rank.STRAIGHT: 4>, two_pair_potential=<Potential.IMPOSSIBLE: 0>, three_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_potential=<Potential.HAS_RANK: 3>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.IMPOSSIBLE: 0>, four_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_flush_potential=<Potential.IMPOSSIBLE: 0>),
  HandFeatures(rank=<Rank.FULL_HOUSE: 6>, two_pair_potential=<Potential.HAS_RANK: 3>, three_of_a_kind_potential=<Potential.HAS_RANK: 3>, straight_potential=<Potential.IMPOSSIBLE: 0>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_house_potential=<Potential.HAS_RANK: 3>, four_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_flush_potential=<Potential.IMPOSSIBLE: 0>),
  HandFeatures(rank=<Rank.ONE_PAIR: 1>, two_pair_potential=<Potential.IMPOSSIBLE: 0>, three_of_a_kind_potential=<Potential.IMPOSSIBLE: 0>, straight_potential=<Potential.IMPOSSIBLE: 0>, flush_potential=<Potential.IMPOSSIBLE: 0>, full_ho

In [ ]:
for hand in environment.hands:
    print(hand)

5♣, 3♠, 6♣, 7♦, 4♦, Q♣, 4♠
7♣, 4♣, 6♣, 7♦, 4♦, Q♣, 4♠
K♦, 9♥, 6♣, 7♦, 4♦, Q♣, 4♠
8♣, 5♠, 6♣, 7♦, 4♦, Q♣, 4♠
